近似求导

In [1]:
# 使用tf.keras搭建回归模型，数据集使用加利福尼亚的房价预测
#导入必要的库即版本

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras
#import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd,sklearn,tf,keras:
    print(module.__name__, module.__version__)

2.0.0-beta1
sys.version_info(major=3, minor=7, micro=0, releaselevel='final', serial=0)
matplotlib 2.2.3
numpy 1.16.4
pandas 0.23.4
sklearn 0.19.2
tensorflow 2.0.0-beta1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
def f(x):
    return 3. * x ** 2 + 2. * x - 1

def approximate_derivative(f, x, eps=1e-3):
    return (f(x + eps) - f(x - eps)) / (2. * eps)

print(approximate_derivative(f, 1.))

7.999999999999119


In [3]:
def g(x1, x2):
    return (x1 + 5) * (x2 ** 2)

def approximate_gradient(g, x1, x2, eps=1e-3):
    dg_x1 = approximate_derivative(lambda x: g(x,x2),x1,eps)
    dg_x2 = approximate_derivative(lambda x: g(x1,x),x2,eps)
    
    return dg_x1,dg_x2

print(approximate_gradient(g,2,3))

(8.999999999993236, 41.999999999994486)


In [4]:
x1 = tf.Variable(2.0)
x2 = tf.Variable(3.0)
with tf.GradientTape(persistent=True) as tape:
    z = g(x1,x2)
    
dz_x1 = tape.gradient(z,x1)
dz_x2 = tape.gradient(z,x2)
print(dz_x1, dz_x2)

del tape

W0814 15:30:20.748661 10328 deprecation.py:323] From D:\SoftWare\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


tf.Tensor(9.0, shape=(), dtype=float32) tf.Tensor(42.0, shape=(), dtype=float32)


In [5]:
x1 = tf.Variable(2.0)
x2 = tf.Variable(3.0)
with tf.GradientTape() as tape:
    z = g(x1,x2)
    
dz_x1x2 = tape.gradient(z,[x1,x2])

print(dz_x1x2)


[<tf.Tensor: id=100, shape=(), dtype=float32, numpy=9.0>, <tf.Tensor: id=112, shape=(), dtype=float32, numpy=42.0>]


In [6]:
# 常量导数/梯度
x1 = tf.constant(2.0)
x2 = tf.constant(3.0)
with tf.GradientTape() as tape:
    tape.watch(x1)
    tape.watch(x2)
    z = g(x1,x2)
    
dz_x1x2 = tape.gradient(z,[x1,x2])

print(dz_x1x2)


[<tf.Tensor: id=136, shape=(), dtype=float32, numpy=9.0>, <tf.Tensor: id=148, shape=(), dtype=float32, numpy=42.0>]


In [7]:
# 一个变量，多个函数求导
x = tf.Variable(5.0)
with tf.GradientTape() as tape:
    z1 = 3*x
    z2 = x**2
    
tape.gradient([z1,z2],x)

<tf.Tensor: id=205, shape=(), dtype=float32, numpy=13.0>

In [8]:
#二阶导数，嵌套tape
x1 = tf.Variable(2.0)
x2 = tf.Variable(3.0)
with tf.GradientTape(persistent=True) as outer_tape:
    with tf.GradientTape(persistent=True) as inner_tape:
        z = g(x1,x2)
    inner_grads = inner_tape.gradient(z,[x1,x2])
outer_grads = [outer_tape.gradient(inner_grad,[x1,x2])
              for inner_grad in inner_grads]

print(outer_grads)
del outer_tape
del inner_tape

[[None, <tf.Tensor: id=268, shape=(), dtype=float32, numpy=6.0>], [<tf.Tensor: id=319, shape=(), dtype=float32, numpy=6.0>, <tf.Tensor: id=304, shape=(), dtype=float32, numpy=14.0>]]


In [11]:
# 梯度下降法
learning_rate = 0.1
x = tf.Variable(0.0)

for _ in range(100):
    with tf.GradientTape() as tape:
        z=f(x)
    dz_dx = tape.gradient(z,x)
    x.assign_sub(learning_rate*dz_dx)
    
print(x)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-0.3333333>


In [12]:
# 与keras中的optimizer结合使用
learning_rate = 0.1
x = tf.Variable(0.0)

optimizer = keras.optimizers.SGD(lr=learning_rate)

for _ in range(100):
    with tf.GradientTape() as tape:
        z=f(x)
    dz_dx = tape.gradient(z,x)
    optimizer.apply_gradients([(dz_dx,x)])# 参数为[(梯度，变量)]
    
print(x)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-0.3333333>
